In [6]:
import sys
sys.path.append("../ralmac")
from registration import *
from correspondence_csv_input import *
from transform_coordinates import *
from merge_dataframe import *
from phantominator import shepp_logan
import warnings
import scipy
# Filter out UserWarning temporarily
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
# size and pixel spacing of the phantom
phantom_shape = (128, 128, 128)
spacing = (1.,) * 3

# create shepp logan test phantom
phantom = shepp_logan(phantom_shape)

## Function to insert lesions inside phantom

def insert_lesion(x,c,r):
    [xx,yy,zz] = np.meshgrid(*[np.arange(y) for y in x.shape])
    lesion = np.zeros(x.shape)
    lesion[((xx-c[0])**2+(yy-c[1])**2+(zz-c[2])**2)<r**2] = 0.5
    return x+lesion

# insert simulated lesions cordinates
#lesion_coords = [(50, 64, 100), (80, 64, 20)]
lesion_coords = [(10, 10, 20), (50, 50, 20)] 
lesion_radii = [3, 8]
lesion_index = [1, 2]
#lesion_plots = [(100, 50), (20, 80)]  # Coordinates are swapped for plotting

for ll, lesions in enumerate(lesion_coords):
    phantom = insert_lesion(phantom,lesion_coords[ll],lesion_radii[ll])

# Convert phantom to SimpleITK format
fixed_image = sitk.GetImageFromArray(phantom)
fixed_image.SetSpacing(spacing)

#### Let us make the moving)image by transformation

# Define a function to apply rotation and translation to the phantom
def transform_phantom(phantom, rotation_angle, translation):
    # Rotate around the 'x' and 'y' axes
    rotated = scipy.ndimage.rotate(phantom, rotation_angle, axes=(0, 1), reshape=False)
    # Apply translation
    translated = scipy.ndimage.shift(rotated, translation)
    return translated

# Apply the transformation to the phantom
rotation_angle = 30  # 30 degrees
translation = (0, 10, 15)  # Translation in z, y, x
transformed_phantom = transform_phantom(phantom, rotation_angle, translation)

# Convert phantom to SimpleITK format
def array_to_sitk(x):
    x_sitk = sitk.GetImageFromArray(x)
    x_sitk.SetOrigin((0, 0, 0))
    x_sitk.SetSpacing(spacing)
    return x_sitk

# Convert transformed phantom to SimpleITK format
moving_image = array_to_sitk(transformed_phantom)

# Define rotation matrices
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta degrees.
    """
    theta = np.radians(theta)
    axis = np.asarray(axis)
    axis = axis / np.sqrt(np.dot(axis, axis))
    a = np.cos(theta / 2.0)
    b, c, d = -axis * np.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa+bb-cc-dd, 2*(bc+ad), 2*(bd-ac)],
                     [2*(bc-ad), aa+cc-bb-dd, 2*(cd+ab)],
                     [2*(bd+ac), 2*(cd-ab), aa+dd-bb-cc]])

# Apply rotation and translation to lesion coordinates
transformed_lesion_coords = []
rotation_matrix_x = rotation_matrix([1, 0, 0], rotation_angle)
rotation_matrix_y = rotation_matrix([0, 1, 0], rotation_angle)

for coord in lesion_coords:
    rotated_coord = np.dot(rotation_matrix_x, np.dot(rotation_matrix_y, coord))
    transformed_coord = rotated_coord + translation
    
    # Convert the transformed coordinate array back to tuple
    transformed_coord_tuple = tuple(transformed_coord)
    
    # Append to the list of transformed coordinates
    transformed_lesion_coords.append(transformed_coord_tuple)

# Display the transformed lesion coordinates
print("Transformed Lesion Coordinates in the Moving Image:")
for i, coord in enumerate(transformed_lesion_coords):
    print(f"Lesion {i + 1}: {coord}")
    
    
def save_lesion_coordinates(lesion_coords, lesion_index, filename):
    # Create a DataFrame to store the lesion coordinates and indices
    df = pd.DataFrame({'x': [coord[0] for coord in lesion_coords],
                       'y': [coord[1] for coord in lesion_coords],
                       'z': [coord[2] for coord in lesion_coords],
                       'Index': [str(index) for index in lesion_index]})
    # Write the DataFrame to a CSV file
    df.to_csv(filename, index=False)
    
    
# Filenames
filename_fixed = 'fixed_coordinates.csv'
filename_moving = 'moving_coordinates.csv'

# Save the fixed lesion coordinates
save_lesion_coordinates(lesion_coords, lesion_index, filename_fixed)

# Save the moving lesion coordinates
save_lesion_coordinates(transformed_lesion_coords, lesion_index, filename_moving)

moving_resampled, [init_transform, final_transform] = registration_3d_rigid_series(fixed_image, moving_image)


final_transform_1 = final_transform.GetInverse()

registered_coordinates = create_transformed_dataframe(filename_moving, final_transform_1)

def save_transformed_dataframe(transformed_df, output_file):
    """Saves the transformed DataFrame to a CSV file."""
    transformed_df.to_csv(output_file, index=False)

# Assuming 'registered_coordinates' is your transformed DataFrame and 'output_file' is the desired filename
save_transformed_dataframe(registered_coordinates, 'registered_coordinates.csv')

threshold = 30  # Set your threshold here
filename_registered = 'registered_coordinates.csv'
# Process correspondences
correspondences, unmatched_names_df1, unmatched_names_df2 = process_lesion_timepoints(filename_fixed, filename_registered, threshold)

plot_lesion_correspondences_timepoints(filename_fixed, filename_registered)
# Create the final DataFrame
final_df = create_final_dataframe_timepoints(correspondences, unmatched_names_df1, unmatched_names_df2, filename_fixed, filename_registered)

df_transformed = merge_indices(final_df)

# Save the resulting DataFrame to a CSV file
df_transformed.to_csv('correspondence_indices.csv', index=False)